# 问题

- 你想在子类中调用父类的某个已经被覆盖的方法。

## 解决方案

- 为了调用父类(超类) 的一个方法，可以使用super() 函数，比如：

In [2]:
class A:
    def spam(self):
        print("A.spam")
        
class B(A):
    def spam(self):
        print("B.spam")
        super().spam()

- super() 函数的一个常见用法是在`__init__()` 方法中确保父类被正确的初始化
了：

In [17]:
class A:
    def __init__(self):
        self.x = 0
        
class B:
    def __init__(self):
        super().__init__()
        self.y = 1

In [18]:
b = B()
b.__dict__

{'y': 1}

- super() 的另外一个常见用法出现在覆盖Python 特殊方法的代码中，比如：

In [14]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj
        
    def __getattr__(self, name):
        return getattr(self._obj, name)
    
    def __setter__(self, name, value):
        if name.startswith('_'):
            super().__setter__(name, value)
        else:
            setattr(self._obj, name, value)
            

- 在上面代码中，`__setattr__()` 的实现包含一个名字检查。如果某个属性名以下划
线(_) 开头，就通过super() 调用原始的`__setattr__() `，否则的话就委派给内部的
代理对象self._obj 去处理。这看上去有点意思，因为就算没有显式的指明某个类的
父类，super() 仍然可以有效的工作。

## 讨论
- 实际上，大家对于在Python 中如何正确使用super() 函数普遍知之甚少。你有时
候会看到像下面这样直接调用父类的一个方法：

In [21]:
class Base:
    def __init__(self):
        print("Base.__init__")

class A(Base):
    def __init__(self):
        Base.__init__(self)
        print("A.__init__")

- 尽管对于大部分代码而言这么做没什么问题，但是在更复杂的涉及到多继承的代
码中就有可能导致很奇怪的问题发生。比如，考虑如下的情况：

In [22]:
class Base:
    def __init__(self):
        print("Base.__init__")
    
class A(Base):
    def __init__(self):
        Base.__init__(self)
        print("A.__init__")
        
class B(Base):
    def __init__(self):
        Base.__init__(self)
        print("B.__init__")
        
class C(A, B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)
        print("C.__init__")

- 如果你运行这段代码就会发现`Base.__init__()` 被调用两次，如下所示：

In [24]:
c = C()

Base.__init__
A.__init__
Base.__init__
B.__init__
C.__init__


- 可能两次调用`Base.__init__()` 没什么坏处，但有时候却不是。另一方面，假设
你在代码中换成使用super() ，结果就很完美了：

In [29]:
class Base:
    def __init__(self):
        print("Base.__init__")
    
class A(Base):
    def __init__(self):
        super().__init__()
        print("A.__init__")
        
class B(Base):
    def __init__(self):
        super().__init__()
        print("B.__init__")
        
class C(A, B):
    def __init__(self):
        super().__init__()
        print("C.__init__")

- 运行这个新版本后，你会发现每个`__init__()` 方法只会被调用一次了：

In [31]:
c = C()

Base.__init__
B.__init__
A.__init__
C.__init__


In [33]:
C.__mro__

(__main__.C, __main__.A, __main__.B, __main__.Base, object)